In [ ]:
# Título del proyecto

'''

Shark adventures: ¿Quieres sentir la adrenalina de nadar con tiburones?

'''

# Objetivo del proyecto

'''
Encontra la ubicación idónea para aquellas que quieren vivir experiencias extremas
'''

In [ ]:
# Realizamos la importaciones que necesitaremos a lo largo del proyecto

import numpy as np
import pandas as pd

In [ ]:
# Importamos el archivo csv

data = pd.read_csv('GSAF5.csv')

In [ ]:
# Guía de trabajo del proyecto
'''
One of the first things we want to do is examine the data and look for any potential issues. Some of the things we are interested in identifying in the data at this stage include:

    Missing values
    Special characters
    Incorrect values
    Extreme values or outliers
    Duplicate records
    Incorrect data types

'''


In [ ]:
# Encontramos y eliminamos registros duplicados
'''
The first thing we will do is attempt to drop any duplicate records, considering all the columns 
we currently have in the data set. Pandas provides us with the ability to do that via the 
drop_duplicates method. We will use the len method to calculate the number of rows in the data set 
both before and after removing duplicates and then print the number of rows dropped.
'''
before = len(data)
data = data.drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))

In [ ]:
# Comenzamos con una perspectiva visual del dataset.

data.head()

In [ ]:
# Renaming columns

'''
Data will often come either without column names or with column names that are not as intuitive 
as they could be. When this is the case, we want to assign descriptive names to the columns so that 
we remember what the values in each column represent. 
'''

data.columns


In [ ]:
# Cambio de varias columnas.

data.columns = ['Manufacturer','Model','Year','Displacement',
                'Cylinders','Transmission','Drivetrain',
                'Vehicle Class','Fuel Type','Fuel Barrels/Year',
                'City MPG','Highway MPG','Combined MPG',
                'CO2 Emission Grams/Mile','Fuel Cost/Year']

data.columns


# Cambio de pocas columnas

data = data.rename(columns={'Manufacturer':'Make',
                            'Displacement':'Engine Displacement'})

data.columns


In [ ]:
# Elección y creación de subset objeto de estudio

'''
When working with data, analysts often need to filter the data based on one or more conditional statements. 

We could enter our conditions inside square brackets to subset the data set for just the records that meet 
the conditions we’ve specified.

'''

filtered = data[(data['Make']=='Ford') & 
                (data['Cylinders']>=6) & 
                (data['Combined MPG'] < 18)]

filtered.head()


In [ ]:
# Cambiando el orden de las columnas

column_order = ['Year','Make','Model','Vehicle Class',
                'Transmission','Drivetrain','Fuel Type',
                'Cylinders','Engine Displacement','Fuel Barrels/Year',
                'City MPG','Highway MPG','Combined MPG',
                'CO2 Emission Grams/Mile','Fuel Cost/Year']

data = data[column_order]
data.head()

In [ ]:
# Crear columnas con categorías condicionales significativas: manual o automática
'''
Another way to create intuitive additional categories in your data is to create columns based on conditional statements. Earlier in this lesson, we filtered our data based on conditional statements. Here, we will populate the values in a column based on them using the loc method.

Our vehicles data set currently has 45 different values in the Transmission field, but one of the key pieces of information embedded in there is whether a vehicle has an automatic or manual transmission. It would be valuable to extract that so that we could group vehicles by their transmission type. Let’s look at how we can create a new TransType column that only contains one of two values for each vehicle: Automatic or Manual.
'''
data.loc[data['Transmission'].str.startswith('A'), 'TransType'] = 'Automatic'
data.loc[data['Transmission'].str.startswith('M'), 'TransType'] = 'Manual'



In [ ]:
# One hot encoding categoriacal variables

'''
This is useful for performing analyses where you want to know whether something falls into a specific 
category or not. It will also be useful when you learn about machine learning, as one-hot encoding makes 
it easier for some algorithms to interpret and find patterns in categorical data.

To perform one-hot encoding on a column, you can use the Pandas get_dummies method and 
pass it the column you would like to one-hot encode.

'''
drivetrain = pd.get_dummies(data['Drivetrain'])
drivetrain.head()

In [ ]:
'''
Missing Values

From this initial view, we can see that our data set contains some columns that have missing values 
in them and others that seem to have a lot of zero values. Let’s see how prevalent missing values are 
in our data. We can use the Pandas isnull method to check whether the value in each field is missing (null) 
and return either True or False for each field. We can use the sum method to total up the number of 
True values by column, and then we can add a condition using square brackets that will filter the data 
and show us only columns where the number of null values were greater than zero.
'''
null_cols = data.isnull().sum()
null_cols[null_cols > 0]


In [ ]:
'''
We can see that some columns have relatively few null values while others have tens of thousands of nulls. 
For fields that have a lot of null values, you will often have to make a judgement call. If you don’t 
think the information is going to be very useful to your analysis, then you would remove those columns 
from your data frame.
'''
drop_cols = list(null_cols[null_cols > 10000].index)
data = data.drop(drop_cols, axis=1)

In [ ]:
# Remaining columns that have null values.

'''
Of the columns that remain, it looks like the cylinders column and the displ column have a similar 
number of nulls. Perhaps they are missing for similar reasons. We can investigate this by subsetting 
the data set and looking at just the records where displ is null and just the columns we think will 
be informative in allowing us to determine a reason.
'''

null_displ = data[(data['displ'].isnull()==True)]
null_displ = null_displ[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]
null_displ




In [ ]:
# Estrategia para eliminar los null values.

'''
We can see that most of the time, cylinders is null when displ is null and that the most of the records 
where both fields are null have a fuel type of Electricity. This makes sense, as electric cars do not 
have cylinders and can therefore not have any displacement. 
In this case, it would make sense to replace these null values with zeros.
 there are other strategies for filling in nulls. Depending on the circumstances, you might want to 
 replace nulls with the column mean or mode values.
'''

data[['displ', 'cylinders']] = data[['displ', 'cylinders']].fillna(0)

In [ ]:
# Incorrect values

'''
in the previous section, we learned that a vehicle without cylinders should not have displacement 
and vice versa. Let’s check to see if there are any cases that violate these rules.
'''

test = data[(data['cylinders']==0) & (data['displ']!=0)]
test[['year', 'make', 'model', 'trany', 'drive','fuelType','cylinders', 'displ']]


In [ ]:
'''
try to look at similar vehicles in the data set and determine the most likely value for this field.

Suppose that using one of the aforementioned methods, we found out that this vehicle actually 
has a 4 cylinder engine. Once we have this information, we can use the loc method to update that 
specific value in the data frame.

Challenge: Try to find other values that might be incorrect in the data set based on what you know about automobiles and correct them.
'''

data.loc[(data['cylinders']==0) & (data['displ']!=0), 'cylinders'] = 4

In [ ]:
# Low variance column

'''
When analyzing data, we want the fields we are working with to be informative, and we will want to 
strip away any columns that don’t have a lot of value to us. One easy way to do this is to identify 
columns that have low variance, where the majority of the values in the column are the same. Since 
there is not a lot of variability in these columns, they have the potential to not be as informative 
as columns that have a variety of different values in them.

Let’s try to identify columns where at least 90% of the values are the same so that we can remove them 
from our data set. 
'''

low_variance = []

for col in data._get_numeric_data():
    minimum = min(data[col])
    ninety_perc = np.percentile(data[col], 90)
    if ninety_perc == minimum:
        low_variance.append(col)


In [ ]:
'''
This returned 34 columns that we could potentially eliminate due to not having high enough variability 
to be informative. Of course, before we do this, we should check the values that do exist in these fields 
to confirm that they are not very informative. Once they have been checked, we can use the the drop method 
like we did earlier in this lesson to remove those columns from our data frame.
'''
data = data.drop(low_variance, axis=1)


In [ ]:
# Extreme values and outliers

'''
hese outliers can influence our aggregations when we are analyzing data later, so we want to make sure 
we address them during our data cleaning stage.

A common method for identifying outliers is one that leverages the interquartile range (IQR). 
Once the IQR is calculated, it is multiplied by a constant (typically 1.5) and lower and upper bounds are 
established at:

    25th Percentile - (IQR x 1.5)
    75th Percentile + (IQR x 1.5)

Any values outside this range are potential outliers and should be investigated.
'''

stats = data.describe().transpose()
stats['IQR'] = stats['75%'] - stats['25%']
stats

In [ ]:
'''
We will then create an empty data frame called outliers with the same columns as our data set. 
Finally, we will loop through each column in the data calculating the lower and upper bounds, 
retrieving records where the value for that column falls outside the bounds we established, 
and appending those results to our outlier data frame.

outliers = pd.DataFrame(columns=data.columns)
'''

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = data[(data[col] < lower) | 
                   (data[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)



In [ ]:
# Binning numeric variables

'''
When preparing data to be analyzed, one of the things that is useful to do is to create additional 
categorical variables. Categorical variables allow you to group records in different ways, 
and each way that you categorize them can provide you with a different perspective when you’re 
conducting your analysis. A common way of creating additional categorical fields is to bin numeric 
variables in a column based on how relatively high or low they are.
'''

mpg_labels = ['Very Low', 'Low', 'Moderate', 'High', 'Very High']




In [ ]:
'''
Next, we must determine how we want our data to be binned. There are three main approaches that we can choose from:

    Equal width bins: the range for each bin is the same size.
    Equal frequency bins: approximately the same number of records in each bin.
    Custom-sized bins: the user explicitly defines where they want the cutoff for each bin to be.

If you want equal width bins, you can use the cut method 

'''

bins = pd.cut(data['Combined MPG'],5, labels=mpg_labels)
bins.head(10)

# Equal frequency bins

bins = pd.qcut(data['Combined MPG'],5, labels=mpg_labels)
bins.head(10)

'''
Note the difference in results. With equal width binning, there will be some bins that contain more 
records than others (such as the Low bin). With equal frequency binning, some of those records will 
be forced into other bins (e.g. the Moderate bin and even the High bin). This is an important consideration 
when determining how you want to categorize your data.
'''

In [ ]:
'''
Finally, if you want custom bin sizes, you can pass a list of bin range values to the cut method instead of the number of bins, and it will bin the values for you accordingly.
'''

cutoffs = [7,14,21,23,30,40]
bins = pd.cut(data['Combined MPG'],cutoffs, labels=mpg_labels)
bins.head(10)


In [ ]:
#Data type correction
'''
This typically occurs when there is a numeric variable that should actually be represented as 
a categorical variable. 
'''
data.dtypes


In [ ]:
'''
Pandas currently has the year column stored as integers, but what if we wanted the year to be stored 
as a categorical variable (object) instead? 
'''
data['year'] = data['year'].astype('object')
data['year'].dtype


In [ ]:
'''
In our vehicles data set, the trany field has several special characters (parentheses, hyphens, etc.). 
We can take a look at the unique values in this column by using the set function.
'''
print(set(data['trany']))


In [ ]:
'''
So let’s remove all hyphens from this column with the help of the str.replace method and then print unique 
values again to ensure they were removed.

'''
data['trany'] = data['trany'].str.replace('-', '')
print(set(data['trany']))

In [ ]:
'''
You will also notice that in some cases Automatic is abbreviated to Auto and in other cases it is spelled out.
We can make that more consistent by using the same technique. While we are at it, let’s also attempt 
to remove parentheses and make spacing more consistent.
'''
data['trany'] = data['trany'].str.replace('Automatic', 'Auto')
data['trany'] = data['trany'].str.replace('Auto\(', 'Auto ')
data['trany'] = data['trany'].str.replace('Manual\(', 'Manual ')
data['trany'] = data['trany'].str.replace('\(', '')
data['trany'] = data['trany'].str.replace('\)', '')


In [ ]:
'''

As you can see, we now have no special characters, consistent naming, and proper spacing. We started out with 47 unique values in this column, and using this technique, we were able to reduce the number of unique values to 39.

'''
print(set(data['trany']))

In [ ]:
# Segunda búsqueda y eliminación de registros duplicados
'''
The first thing we will do is attempt to drop any duplicate records, considering all the columns 
we currently have in the data set. Pandas provides us with the ability to do that via the 
drop_duplicates method. We will use the len method to calculate the number of rows in the data set 
both before and after removing duplicates and then print the number of rows dropped.
'''
before = len(data)
data = data.drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))

In [ ]:
# Total price of all houses sold
house_df['SalePrice'].sum()

1901400

# Average lot size of houses sold
house_df['LotSize'].mean()

10123.1

# The latest year a house in the data set was built
house_df['YearBuilt'].max()

2004

# The eariliest year a house in the data set was built
house_df['YearBuilt'].min()

In [ ]:
# Combinación de subsets: merge

'''
Another useful thing to do with data sets is to combine them. Pandas provides us with a few different 
ways to do this. The first way is by merging. Merging is similar to creating a join in SQL, 
where you can specify common fields between the two tables and then include information from both 
in your query. Pandas has a merge method that functions in a similar way.

To illustrate, let’s create a data frame that has the average Combined MPG for each Make using the 
groupby method. We will merge that average into our data frame, joining on Make, so that we can see 
how fuel efficient a vehicle is in comparison to the other vehicles made by the same manufacturer.

'''

avg_mpg = data.groupby('Make', as_index=False).agg({'Combined MPG':'mean'})
avg_mpg.columns = ['Make', 'Avg_MPG']

data = pd.merge(data, avg_mpg, on='Make')
data.head(10)



In [ ]:
# Combinación de subsets: concat

'''
The pandas concat method lets you attach columns or rows from one data set onto another data set as 
long as both data sets have the same number of rows (if you are concatenating columns) or columns 
(if you are concatenating rows). Let’s take a look at examples for each of these.

For column concatenation, we can use the one-hot encoded drivetrain data frame we created earlier 
and add those columns to our vehicles data set. Note that the data frames passed to the concat 
method must be in a list and you set the axis parameter to 1 in order to indicate that you are 
concatenating columns.
'''
data = pd.concat([data, drivetrain], axis=1)
'''
To illustrate row concatenation, let’s create two new data frames based on conditional filters 
from our original data frame - one containing only Lexus vehicles and another containing only Audi vehicles. 
We will then combine them using the concat method into a lexus_audi data frame that contains only vehicles 
manufactured by those two companies.
'''
lexus = data[data['Make']=='Lexus']
audi = data[data['Make']=='Audi']

lexus_audi = pd.concat([lexus, audi], axis=0)
'''
Again, note that the data frames are passed as a list and that this time the axis is set to 0 to specify that we are concatenating rows.
'''

In [ ]:
# Combinación de subset: agrupando por variable

'''
Our vehicles data set currently has a wide format, where there is a column for each attribute. 
However, some analytic and visualization tasks will require that the data be in a long format, 
where there are a few variables that define the entities and then all other attribute information 
is condensed into two columns: one containing the column/attribute names and another containing the value 
for that attribute for each entity. Pandas makes it easy to format data this way with the melt function. 
For example, suppose we were going to perform some analysis or visualization task where we needed the Year, 
Make, and Model to identify the vehicles and then we also needed the City MPG, Highway MPG, and Combined MPG 
fields for performing various calculations. Below is how we would melt the data into the proper format.

'''

melted = pd.melt(data, id_vars=['Year','Make','Model'], 
                 value_vars=['City MPG','Highway MPG','Combined MPG'])
melted.head(10)

'''
As you can see, the column names have been stacked into the the variable field and their 
corresponding values have been stacked into the value field.
'''

In [ ]:
# Exporting DataFrame

# Export comma-separated variable file
data = pd.to_csv('vehicles/vehicles.csv', index=False)